In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import re

In [2]:
data = pd.read_csv('feedback_dataset.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5200 entries, 0 to 5199
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       5200 non-null   object
 1   sentiment  5200 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 81.4+ KB


In [4]:
def to_sentiment(sentiment):
  sentiment = sentiment
  if sentiment == 0:
    return 'negative'
  else: 
    return 'positive'
data['sentiment'] = data.sentiment.apply(to_sentiment)

In [5]:
def preProcess_data(text):
   text = text.lower()
   new_text = re.sub('[^a-zA-z0-9\s]','',text)
   new_text = re.sub('rt', '', new_text)
   return new_text

data['text'] = data['text'].apply(preProcess_data)

In [6]:
max_fatures = 2000

tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X, 28) 

Y = pd.get_dummies(data['sentiment']).values

In [7]:
Y

array([[0, 1],
       [0, 1],
       [0, 1],
       ...,
       [1, 0],
       [1, 0],
       [1, 0]], dtype=uint8)

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20)

In [9]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.3, recurrent_dropout=0.2, return_sequences=True))
model.add(LSTM(128,recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [10]:
batch_size = 512

model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, validation_data=(X_test, Y_test))

Epoch 1/15
9/9 [==============================] - 22s 1s/step - loss: 0.6730 - accuracy: 0.5591 - val_loss: 0.6581 - val_accuracy: 0.5933
Epoch 2/15
9/9 [==============================] - 10s 1s/step - loss: 0.6311 - accuracy: 0.6327 - val_loss: 0.6364 - val_accuracy: 0.6154
Epoch 3/15
9/9 [==============================] - 10s 1s/step - loss: 0.5859 - accuracy: 0.6733 - val_loss: 0.5914 - val_accuracy: 0.6683
Epoch 4/15
9/9 [==============================] - 9s 1s/step - loss: 0.5019 - accuracy: 0.7572 - val_loss: 0.4907 - val_accuracy: 0.7596
Epoch 5/15
9/9 [==============================] - 10s 1s/step - loss: 0.3858 - accuracy: 0.8373 - val_loss: 0.4231 - val_accuracy: 0.8260
Epoch 6/15
9/9 [==============================] - 10s 1s/step - loss: 0.3184 - accuracy: 0.8726 - val_loss: 0.3945 - val_accuracy: 0.8356
Epoch 7/15
9/9 [==============================] - 10s 1s/step - loss: 0.2667 - accuracy: 0.8959 - val_loss: 0.3660 - val_accuracy: 0.8462
Epoch 8/15
9/9 [===================

In [11]:
# we have saved our model in ‘hdf5’ format
model.save('sentiment.h5')